In [40]:
#　def
import networkx as nx
import numpy as np
import ast
def compute_information_centrality_by_Laplacian(G: nx.Graph, node):
    L = nx.laplacian_matrix(G).toarray()
    n = G.number_of_nodes()
    L_dagger = np.linalg.pinv(L)
    effective_resistance_of_node = n * L_dagger[node][node] + np.trace(L_dagger)
    information_centrality = n / effective_resistance_of_node
    return information_centrality

In [41]:
# example for karate
input_file = "../../data/karate.mtx"
with open(input_file, 'r') as file:
    edges = [(int(a) - 1, int(b) - 1) for a, b in (line.split() for line in file)]
G = nx.Graph()
G.add_edges_from(edges)

In [42]:
# Read result
target_nodes = []
edge_need_remove = []
result_file = "../../result/karate_result.txt"

with open(result_file, "r") as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith("Target nodes:"):
            target_nodes = ast.literal_eval(line.split(":", 1)[1].strip())
        elif line.startswith("Edges to remove per target node:"):
            edge_lines = lines[lines.index(line) + 2:]
            edge_str = ''.join([l.strip() for l in edge_lines])
            edge_str = edge_str.rstrip(',')
            edge_str = edge_str.rstrip(']') + ']'
            edges = ast.literal_eval(edge_str.strip())
            edge_need_remove.extend(edges)
# print("Target nodes:", target_nodes)
# print("Edges to remove per target node:", edge_need_remove)
pair_target_edges = []
for idx, target in enumerate(target_nodes):
    pair_target_edges.append([target, edge_need_remove[idx]])

In [45]:
# Calculate score now
sum_of_information_centrality = 0
sum_of_information_centrality_origin = 0
for target, edge_list in pair_target_edges:
    A = G.copy()
    A.remove_edges_from(edge_list)
    information_centrality = compute_information_centrality_by_Laplacian(A, target)
    information_centrality_origin = compute_information_centrality_by_Laplacian(G, target)
    sum_of_information_centrality += information_centrality
    sum_of_information_centrality_origin += information_centrality_origin
average = sum_of_information_centrality / len(pair_target_edges)
average_origin = sum_of_information_centrality_origin / len(pair_target_edges)
print("Information centrality of targets: ", average_origin)
print("Information centrality of targets: ", average)

Information centrality of targets:  1.423238131167291
Information centrality of targets:  0.8222515330010772
